In [31]:
%%writefile custom_script.py

import joblib
import argparse
from CustomtestModel import CustomtestModel

def main(inputfile):
    model = CustomtestModel(inputfile = inputfile)
    response = model.getjson()
    model_file_name = 'test_custom_model'
    joblib.dump(response, model_file_name)
    
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--inuptfilepath", type=str, default="testdata.xlsx")
    args = parser.parse_args()
    main(inputfile=args.inuptfilepath)    

Overwriting custom_script.py


In [32]:
! python custom_script.py

In [33]:
import joblib

model_file_name = "test_custom_model"

mymodel = joblib.load(model_file_name)
mymodel

{'test': {'testdata': [101], 'realdata': [201]}}

In [34]:
!tar czvf testcustommodel.tar.gz $model_file_name

test_custom_model


In [35]:
import sagemaker

sess = sagemaker.Session()
sess.default_bucket()

'sagemaker-us-east-2-054898670646'

In [36]:
import boto3

botosession = boto3.Session()
s3 = botosession.resource("s3")
client = boto3.client("s3")

In [37]:
import os

prefix = "umasankar/"
bucket = "sagemaker-us-east-2-054898670646"

fobj = open("testcustommodel.tar.gz", "rb")
key = os.path.join("Models",prefix,model_file_name,"testcustommodel.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fobj)

Models/umasankar/test_custom_model/testcustommodel.tar.gz


In [38]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "1.3-1")
container

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.3-1'

In [39]:
role = sagemaker.get_execution_role()
role

'arn:aws:iam::054898670646:role/service-role/AmazonSageMaker-ExecutionRole-20230114T184345'

In [40]:

model_name = "custommodeltest2"
model_url = "s3://sagemaker-us-east-2-054898670646/Models/umasankar/test_custom_model/testcustommodel.tar.gz"
sm_client = boto3.client("sagemaker")

primary_container = {
    "Image" : container,
    "ModelDataUrl": model_url
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response)
print(create_model_response["ModelArn"])

{'ModelArn': 'arn:aws:sagemaker:us-east-2:054898670646:model/custommodeltest2', 'ResponseMetadata': {'RequestId': '40e25fb1-4530-4c76-8356-40ff3e2aaeae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '40e25fb1-4530-4c76-8356-40ff3e2aaeae', 'content-type': 'application/x-amz-json-1.1', 'content-length': '78', 'date': 'Sun, 22 Jan 2023 02:29:11 GMT'}, 'RetryAttempts': 0}}
arn:aws:sagemaker:us-east-2:054898670646:model/custommodeltest2


In [41]:
# endpoint configuration

endpoint_config_name = "custommodelendpointconfig1"

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants = [
        {
            "InstanceType":"ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic"
        }
    ]
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint Config Arn: arn:aws:sagemaker:us-east-2:054898670646:endpoint-config/custommodelendpointconfig1


In [42]:
endpoint_name = "custommodelendpointtest1"

create_endpoint_response = sm_client.create_endpoint(
    EndpointName = endpoint_name, EndpointConfigName=endpoint_config_name
)

print(create_endpoint_response["EndpointArn"])

arn:aws:sagemaker:us-east-2:054898670646:endpoint/custommodelendpointtest1


In [ ]:
import sagemaker

In [4]:
import boto3
sm_client = boto3.client("sagemaker")
sm_client.delete_endpoint(EndpointName="custommodelendpointtest1")









{'ResponseMetadata': {'RequestId': 'd0121f79-14ee-43f9-a79c-f4f2736089ac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd0121f79-14ee-43f9-a79c-f4f2736089ac',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 22 Jan 2023 06:19:47 GMT'},
  'RetryAttempts': 0}}